In [1]:
import pandas as pd
import numpy as np
import gensim
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
# from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from utils import save_object, load_object
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MARK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MARK\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\MARK\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MARK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\MARK\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\MARK\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
messages = pd.read_csv('D:/PROJECTS/Spam_ham/notebooks/data/SMSSpamCollection.txt', sep='\t', names=['label', 'message'])

In [3]:
messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
messages.shape

(5572, 2)

In [5]:
lemmatizer = WordNetLemmatizer()

#### Determine X an y variables

In [6]:
# X = messages.loc[:, ['message']]
# y = messages.loc[:,['label']]
X = messages['message']
y = messages['label']

In [7]:
X

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: message, Length: 5572, dtype: object

In [9]:
y

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: label, Length: 5572, dtype: object

### Data Cleaning and lemmatizing

In [10]:
lemmatizer = WordNetLemmatizer()

In [11]:
def clean_text(raw_text):
    
    corpus = []
    sw_list = stopwords.words('english')
    for i in range(0,len(raw_text)):
        review = re.sub('[^a-zA-Z]0-9', ' ',raw_text[i])
        review = review.lower()
        review = review.split()
        review = [lemmatizer.lemmatize(word) for word in review if word not in sw_list]
        review = ' '.join(review)
        corpus.append(review)
    return corpus

In [12]:
corpus = clean_text(X)

In [13]:
corpus

['go jurong point, crazy.. available bugis n great world la e buffet... cine got amore wat...',
 'ok lar... joking wif u oni...',
 "free entry 2 wkly comp win fa cup final tkts 21st may 2005. text fa 87121 receive entry question(std txt rate)t&c's apply 08452810075over18's",
 'u dun say early hor... u c already say...',
 'nah think go usf, life around though',
 "freemsg hey darling 3 week's word back! like fun still? tb ok! xxx std chgs send, £1.50 rcv",
 'even brother like speak me. treat like aid patent.',
 "per request 'melle melle (oru minnaminunginte nurungu vettam)' set callertune callers. press *9 copy friend callertune",
 'winner!! valued network customer selected receivea £900 prize reward! claim call 09061701461. claim code kl341. valid 12 hour only.',
 'mobile 11 month more? u r entitled update latest colour mobile camera free! call mobile update co free 08002986030',
 'gonna home soon want talk stuff anymore tonight, k? cried enough today.',
 'six chance win cash! 100 20,00

### Tokenize the Data

In [14]:
def Word_token(a_corpus):
    words = []
    for sent in a_corpus:
        sent_token = sent_tokenize(sent)
        # print(sent_token)
        # for sent in sent_token:
        words.append(simple_preprocess(sent))
        # words_fixed = [' '.join(word) for word in words]
    return words


In [15]:
words = Word_token(corpus)

In [16]:
words

[['go',
  'jurong',
  'point',
  'crazy',
  'available',
  'bugis',
  'great',
  'world',
  'la',
  'buffet',
  'cine',
  'got',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'oni'],
 ['free',
  'entry',
  'wkly',
  'comp',
  'win',
  'fa',
  'cup',
  'final',
  'tkts',
  'st',
  'may',
  'text',
  'fa',
  'receive',
  'entry',
  'question',
  'std',
  'txt',
  'rate',
  'apply',
  'over'],
 ['dun', 'say', 'early', 'hor', 'already', 'say'],
 ['nah', 'think', 'go', 'usf', 'life', 'around', 'though'],
 ['freemsg',
  'hey',
  'darling',
  'week',
  'word',
  'back',
  'like',
  'fun',
  'still',
  'tb',
  'ok',
  'xxx',
  'std',
  'chgs',
  'send',
  'rcv'],
 ['even', 'brother', 'like', 'speak', 'me', 'treat', 'like', 'aid', 'patent'],
 ['per',
  'request',
  'melle',
  'melle',
  'oru',
  'minnaminunginte',
  'nurungu',
  'vettam',
  'set',
  'callertune',
  'callers',
  'press',
  'copy',
  'friend',
  'callertune'],
 ['winner',
  'valued',
  'network',
  'customer',
  'selected'

#### Apply word2vec on tokenized corpus

In [17]:
model = gensim.models.Word2Vec(words, window=5, min_count=2)

In [18]:
model.wv

In [19]:
model.wv.index_to_key

['call',
 'get',
 'ur',
 'gt',
 'lt',
 'go',
 'ok',
 'free',
 'day',
 'know',
 'now',
 'like',
 'come',
 'good',
 'got',
 'time',
 'you',
 'love',
 'want',
 'text',
 'send',
 'me',
 'it',
 'need',
 'txt',
 'one',
 'today',
 'going',
 'home',
 'stop',
 'lor',
 'sorry',
 'see',
 'still',
 'take',
 'mobile',
 'back',
 'da',
 'dont',
 'reply',
 'think',
 'tell',
 'hi',
 'phone',
 'week',
 'new',
 'later',
 'please',
 'pls',
 'co',
 'make',
 'dear',
 'msg',
 'no',
 'well',
 'say',
 'night',
 'much',
 'that',
 'oh',
 'great',
 'claim',
 'hope',
 'hey',
 'message',
 'give',
 'happy',
 'number',
 'wat',
 'work',
 'thing',
 'min',
 'yes',
 'way',
 'www',
 'there',
 'prize',
 'let',
 'right',
 'tomorrow',
 'already',
 'friend',
 'said',
 'ask',
 'win',
 'cash',
 'amp',
 'im',
 'yeah',
 'life',
 'really',
 'can',
 'meet',
 'babe',
 'tone',
 'find',
 'miss',
 'morning',
 'thanks',
 'last',
 'uk',
 'com',
 'care',
 'would',
 'anything',
 'service',
 'lol',
 'nokia',
 'also',
 'every',
 'feel',
 'ke

In [20]:
len(model.wv.index_to_key)

3729

In [21]:
model.corpus_count

5572

In [22]:
model.epochs

5

In [23]:
model.wv.similar_by_word('kid')

[('next', 0.9974592924118042),
 ('say', 0.9974145889282227),
 ('good', 0.9973961114883423),
 ('love', 0.9973698854446411),
 ('got', 0.9973658323287964),
 ('come', 0.9973442554473877),
 ('night', 0.9973124265670776),
 ('im', 0.9973106384277344),
 ('that', 0.9973065257072449),
 ('keep', 0.9972859025001526)]

In [24]:
model.wv.similar_by_word('prize')

[('claim', 0.9993930459022522),
 ('line', 0.9993138313293457),
 ('call', 0.9993022084236145),
 ('draw', 0.9992403984069824),
 ('show', 0.9992282390594482),
 ('cash', 0.9991888403892517),
 ('mobile', 0.9991136789321899),
 ('contact', 0.9990552067756653),
 ('urgent', 0.9990540146827698),
 ('number', 0.9990144968032837)]

In [25]:
model.wv.similar_by_word('unsubscribe')

[('now', 0.9987550973892212),
 ('stop', 0.998725950717926),
 ('com', 0.9986688494682312),
 ('free', 0.9986395239830017),
 ('phone', 0.998635470867157),
 ('reply', 0.9986212253570557),
 ('msg', 0.9986057281494141),
 ('help', 0.9985731244087219),
 ('chat', 0.9985629916191101),
 ('text', 0.9985508918762207)]

In [26]:
model.wv.similar_by_word('content')

[('find', 0.998650848865509),
 ('place', 0.9986501336097717),
 ('im', 0.9986280798912048),
 ('you', 0.9986211657524109),
 ('her', 0.9985846281051636),
 ('great', 0.9985572099685669),
 ('amp', 0.9985544681549072),
 ('co', 0.9985511302947998),
 ('one', 0.998542070388794),
 ('told', 0.9985396862030029)]

In [27]:
model.wv['unsubscribe'].shape

(100,)

### Convert tokenized words to arrays 

In [28]:
def avg_word2vec(doc, vector_size=100):
    X = []
    valid_words = [word for word in doc if word in model.wv.index_to_key]
    if not valid_words:
        return np.zeros(vector_size)
    X.append(np.mean([model.wv[word] for word in valid_words], axis=0))
    return X

In [29]:
def word_arr(a_words):
    X = []
    for i in tqdm(range(len(a_words))):
        X.append(avg_word2vec(a_words[i]))
    return X

In [30]:
X_array = word_arr(words)

100%|██████████| 5572/5572 [00:00<00:00, 6166.78it/s]


In [37]:
X_array

[[array([-0.11910331,  0.22305155,  0.06281137,  0.00483394, -0.00559855,
         -0.41118744,  0.09423047,  0.51467997, -0.13875683, -0.12798624,
         -0.10697038, -0.35181797, -0.04002907,  0.08043657,  0.07699635,
         -0.20693733, -0.02071005, -0.3844469 ,  0.00709547, -0.4323686 ,
          0.17768733,  0.1576649 ,  0.17691286, -0.12790047, -0.14026633,
          0.02316016, -0.18139668, -0.21378748, -0.17100775,  0.00429598,
          0.2804354 ,  0.04212402,  0.05904584, -0.14306472, -0.14868934,
          0.3067772 ,  0.07866943, -0.23717912, -0.1769365 , -0.4551048 ,
          0.05087191, -0.19960986, -0.09306866, -0.04324284,  0.2262155 ,
         -0.17237069, -0.13718106, -0.06516308,  0.1482559 ,  0.15275422,
          0.09582225, -0.21623279, -0.06743531, -0.09681404, -0.19653283,
          0.15767328,  0.21558021, -0.03640717, -0.22793692,  0.09311069,
          0.09435616,  0.12856252, -0.15933503, -0.03557689, -0.36306012,
          0.2026052 ,  0.08837783,  0.

In [39]:
X_array[0].shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
X_new = np.array(X_array)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (5572,) + inhomogeneous part.

In [35]:
X_new[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [79]:
X_new[0].shape

(100,)

In [80]:
words[0]

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'got',
 'amore',
 'wat']

In [32]:
X.shape

(5572,)

#### Encode y values

In [82]:
encoder = LabelEncoder()

y_array = encoder.fit_transform(y)

In [32]:
y_array

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [33]:
len(y_array)

5572

In [34]:
len(X_array)

5572

#### Split X and y into training and testing

In [35]:
X_train,X_test,y_train,y_test = train_test_split(X_array,y_array,test_size=0.2,random_state=42)

In [36]:
len(X_train)

4457

In [37]:
len(y_train)

4457

In [38]:
from sklearn.metrics import precision_score, accuracy_score, confusion_matrix

In [120]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)

In [39]:
def evaluate_model(true, predicted):
    precision = precision_score(true, predicted)
    conf_mat = confusion_matrix(true, predicted)
    accuracy = accuracy_score(true, predicted)
    return precision, conf_mat, accuracy

In [ ]:
models = {
    'Gaussian Naive-Bayes': GaussianNB(),
    # 'K-Neighbors Regressor': KNeighborsClassifier(),
    'Decision Tree Classifier': DecisionTreeClassifier(),#(random_state=42),
    'Random Forest Classifier': RandomForestClassifier(),#(random_state=42),
    'Gradient Boosting Classifier': GradientBoostingClassifier(),
    # 'CatBoost Regressor Classifier': CatBoostClassifier(verbose=False),
    'AdaBoost Regressor Classifier': AdaBoostClassifier(),#(random_state=42)
}


model_list = []
accuracy_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    # Train the model
    model.fit(X_train, y_train) 
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    #Evaluate train and test dataset
    precision_train, conf_mat_train, accuracy_train = evaluate_model(y_train, y_train_pred)
    precision_test, conf_mat_test, accuracy_test = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for training set')
    print(f'-Precision_score: {precision_train:.4f}.')
    print(f'-Confusion Matrix: {conf_mat_train}.')
    print(f'-accuracy_Score: {accuracy_train:.4f}.')
    
    print('----------------------------------------------------------')
    print('Model performance for test set')
    print(f'-Precision_score: {precision_test:.4f}.')
    print(f'-Confusion Matrix: {conf_mat_test}.')
    print(f'-accuracy_Score: {accuracy_test:.4f}.')

    accuracy_list.append(accuracy_test)
    
    print('='*35)
    print('\n')
    

Gaussian Naive-Bayes
Model performance for training set
-Precision_score: 0.1804.
-Confusion Matrix: [[1733 2126]
 [ 130  468]].
-accuracy_Score: 0.4938.
----------------------------------------------------------
Model performance for test set
-Precision_score: 0.1832.
-Confusion Matrix: [[431 535]
 [ 29 120]].
-accuracy_Score: 0.4942.


Decision Tree Classifier
Model performance for training set
-Precision_score: 1.0000.
-Confusion Matrix: [[3859    0]
 [   0  598]].
-accuracy_Score: 1.0000.
----------------------------------------------------------
Model performance for test set
-Precision_score: 0.7419.
-Confusion Matrix: [[926  40]
 [ 34 115]].
-accuracy_Score: 0.9336.


Random Forest Classifier
Model performance for training set
-Precision_score: 1.0000.
-Confusion Matrix: [[3859    0]
 [   0  598]].
-accuracy_Score: 1.0000.
----------------------------------------------------------
Model performance for test set
-Precision_score: 0.9206.
-Confusion Matrix: [[956  10]
 [ 33 116]].

In [ ]:
pd.DataFrame(list(zip(model_list, accuracy_list)), columns=['Model name', 'Accuracy_score']).sort_values(by=['Accuracy_score'], ascending=False)

,Model name,R2_score
4,CatBoost Regressor Classifier,0.964126
3,Gradient Boosting Classifier,0.963229
2,Random Forest Classifier,0.961435
5,AdaBoost Regressor Classifier,0.942601
1,Decision Tree Classifier,0.933632
0,Gaussian Naive-Bayes,0.494170


In [ ]:
import os
print(os.getcwd)